In [95]:
#preprocess
import json
import nltk
from nltk.stem.porter import PorterStemmer
#model and training
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
#chatbot
import random
#from nltk_utils import bag_of_words, tokenize

import warnings

warnings.filterwarnings("ignore")

In [96]:
#nltk utils

stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)


def stem(word):
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(all_words), dtype=np.float32)
    for index, word in enumerate(all_words):
        if word in tokenized_sentence:
            bag[index] = 1.0
    return bag

In [97]:
#preprocessing training data
with open('myIntents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
sentences = []
#collecting words with corresponding tags
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        sentences.append(pattern)
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))
ignore_words = ['.', '!', ',', '?']
#stemming all_words / removing ignore words
all_words = [stem(w) for w in all_words if w not in ignore_words]
#sorting all_words alphabetically
all_words = sorted(set(all_words))
#sorting tags alphabetically
tags = sorted(set(tags))

In [98]:
#preparing training data
x_train = []
y_train = []

for(pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    x_train.append(bag)
    
    label = tags.index(tag)
    y_train.append(label)

x_train = np.array(x_train)
y_train = np.array(y_train)

#creating pytorch dataset
class ChatDataset(Dataset):
    
    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train
        
    #dataset[index]
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples

In [99]:
#pytorch model

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_of_classes):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size)
        self.layer3 = nn.Linear(hidden_size, num_of_classes)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        output = self.layer1(x)
        output = self.relu(output)
        output = self.layer2(output)
        output = self.relu(output)
        output = self.layer3(output)
        #no activation and no softmax (included in CrossEntropyLoss which will be used later)
        return output
        

In [89]:
#Hyperparameters
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(x_train[0])
learning_rate = 0.001
num_of_epochs = 1000

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)

#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_of_epochs):
    for(words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device,  dtype=torch.long)
        
        #forward
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        #backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if(epoch + 1) % 100 == 0:
        print(f'epoch {epoch+1}/{num_of_epochs}, loss={loss.item():.4f}')
print(f'final loss, loss={loss.item():.4f}')

epoch 100/1000, loss=0.6069
epoch 200/1000, loss=0.2712
epoch 300/1000, loss=0.1430
epoch 400/1000, loss=0.3244
epoch 500/1000, loss=0.7452
epoch 600/1000, loss=0.0000
epoch 700/1000, loss=0.0716
epoch 800/1000, loss=1.1047
epoch 900/1000, loss=1.0377
epoch 1000/1000, loss=0.0000
final loss, loss=0.0000


In [90]:
#save the model
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'Training complete. File saved to {FILE}')

Training complete. File saved to data.pth


In [91]:
#using the saved model

FILE = "data.pth"
data = torch.load(FILE)

model_state = data["model_state"]

loaded_model = NeuralNet(input_size, hidden_size, output_size).to(device)
loaded_model.load_state_dict(model_state)
loaded_model.eval()

NeuralNet(
  (layer1): Linear(in_features=585, out_features=8, bias=True)
  (layer2): Linear(in_features=8, out_features=8, bias=True)
  (layer3): Linear(in_features=8, out_features=75, bias=True)
  (relu): ReLU()
)

In [92]:
#constructing our chatbot

bot_name = "Yoda"
print("Chat, let's do! 'I have the force' to exit, type you must")
while True:
    sentence = input('Young Jedi: ')
    if sentence == "I have the force":
        break
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X)
    
    output = loaded_model(X)
    _, predicted = torch.max(output, dim=1)
    tag = tags[predicted.item()]
    
    #calculating probabilities
    probs = torch.softmax(output, dim=1)
    actual_prob = probs[0][predicted.item()]
    
    if actual_prob.item() > 0.75:
        for intent in intents["intents"]:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: Not wise enough, you are yet.")


Chat, let's do! 'I have the force' to exit, type you must
Young Jedi: hello
Yoda: Hi! How are you doing?
Young Jedi: very nice how bout you?
Yoda: Not wise enough, you are yet.
Young Jedi: game fact
Yoda: The death screams of the bad guys in the original Golden Axe are digitized from screams used in the films First Blood and Conan The Barbarian.
Young Jedi: wow
Yoda: Not wise enough, you are yet.
Young Jedi: you are smart
Yoda: Not wise enough, you are yet.
Young Jedi: who are you
Yoda: Not wise enough, you are yet.
Young Jedi: what is your name
Yoda: You can call me DF.
Young Jedi: you are very smart person
Yoda: You're pretty smart yourself.
Young Jedi: thank you
Yoda: Not wise enough, you are yet.
Young Jedi: thanks
Yoda: No problem!
Young Jedi: can you tell me what you can do?
Yoda: I'm a Chatbot available to help you 24x7
Young Jedi: who is owner?
Yoda: Not wise enough, you are yet.
Young Jedi: who is your owner
Yoda: Not wise enough, you are yet.
Young Jedi: very good
Yoda: Thank

KeyboardInterrupt: Interrupted by user